In [ ]:
import os
import glob
import shutil
from werkzeug.utils import secure_filename
from flask import Flask, request, jsonify, send_from_directory,  render_template, redirect, url_for, session
from os.path import join, dirname, realpath
from tensorflow import keras
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from flask_mysqldb import MySQL,MySQLdb
from uuid import uuid1

app = Flask(__name__,template_folder='./template')
app._static_folder = './static'
app.config["PPHOTO_UPLOADS"] = r'./static/PPHOTO'
app.config["SIGN_UPLOADS"] = r'./static/SIGN'
app.config["ALLOWED_IMAGE_EXTENSIONS"] = ["JPEG", "JPG", "PNG", "GIF","JFIF"]

model_path = './m'
model = tf.keras.models.load_model(model_path)

app.config['MYSQL_HOST']="localhost"
app.config['MYSQL_USER']="root"
app.config['MYSQL_PASSWORD']="2002"
app.config['MYSQL_DB']="information"

mysql=MySQL(app)

@app.route('/')
def home():
    return render_template('index.html')

def allowed_image(filename):

    if not "." in filename:
        return False

    ext = filename.rsplit(".", 1)[1]

    if ext.upper() in app.config["ALLOWED_IMAGE_EXTENSIONS"]:
        return True
    else:
        return False


@app.route('/submit', methods=["GET", "POST"])
def submit():
    
    ##INFORMATION--GET
    NAME = request.form['First_Name']
    FNAME = request.form['fathers_Name']
    DOB = request.form['DOB']
    EMAIL = request.form['Email_Id']
    NUMBER = request.form['Mobile_Number']
    
    
    ##PHOTO--GET--STORE
    if request.files:
        
        PPHOTO = request.files["image"]
        SIGN = request.files["sign"]
        
        if PPHOTO.filename == "":
            print("No filename")
        if SIGN.filename == "":
            print("No filename")
        
        if allowed_image(PPHOTO.filename):
            P_filename = secure_filename(uuid1().__str__() + PPHOTO.filename)
            PPHOTO.save(os.path.join(app.config["PPHOTO_UPLOADS"], P_filename))
            PPHOTO_path =  './static/PPHOTO/{}'.format(P_filename)
            print("Image saved")
        else:
            print("this extension not allowed")
        
        if allowed_image(SIGN.filename):
            filename = secure_filename(uuid1().__str__() + SIGN.filename)
            SIGN.save(os.path.join(app.config["SIGN_UPLOADS"], filename))
            SIGN_path =  './static/SIGN/{}'.format(filename)
            print("Image saved")
        else:
            print("this extension not allowed")
    
    else:
        print('error')
    
    
    
    ##AI
    path = PPHOTO_path
    def finds():
        test_image = image.load_img(path, target_size = (64, 64))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = model.predict(test_image)
        return result
    
    val = finds()
    if  val[0][0] == 1:
              prediction = 'SIGN'
    else:
              prediction = 'PPHOTO'
    
    print(prediction)
    if prediction == 'SIGN':
        src_dir = "./static/PPHOTO/"
        dst_dir = "./static/SIGN/"
        SIGN_path = dst_dir + P_filename
        shutil.move(src_dir + P_filename, dst_dir)
        
        src_dir = "./static/SIGN/"
        dst_dir = "./static/PPHOTO/"
        PPHOTO_path = dst_dir + filename
        shutil.move(src_dir + filename, dst_dir)
              
    ##DATABASE
    cur=mysql.connection.cursor()
    cur.execute("INSERT INTO details (NAME,FNAME,DOB,EMAIL,NUMBER,PPHOTO,SIGN) VALUES (%s,%s,%s,%s,%s,%s,%s)",(NAME,FNAME,DOB,EMAIL,NUMBER,PPHOTO_path,SIGN_path))
    mysql.connection.commit()
    cur.close()

    return render_template('message.html')
    # return render_template('index.html')

@app.route('/result', methods=["GET", "POST"])
def result():
    return render_template('result.html')

@app.route('/view', methods=["GET", "POST"])
def view():
  
    x = request.form['em']
    cur = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
    cur.execute('SELECT * FROM details where EMAIL= %s',(x,))
    output = cur.fetchall()
    return render_template('nhk.html',output=output)


if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
